# **Function Root Estimator**
## **Contents**
1. [Introduction](#Introduction)
2. [The Newton-Raphson Method](#Brownian-Motion-Simulator)
3. [The Secant Method](#Brownian-Motion-Parameter-Dependent-Simulator)
4. [The Bisection Method](#Geometric-Brownian-Motion-Simulator)
5. [Credits](#Credits)

## **Introduction**
This project aims to implement standard numerical root-finding techniques in Python, using the `numpy` package. The methods under consideration are the Newton-Raphson method, the Secant Method and the Bisection Method. The `time` library is also imported in order to monitor computation times.

In [14]:
import numpy as np
from time import time

def performance_metrics(*args, method):
    start = time()
    x = method(*args)
    end = time()
    s = round(end - start, 3)
    f = args[0]
    n = args[-2]
    d = args[-1]
    print(f"Function: {f}", f"Method: {method.__name__}", f"Result: {x}", f"Iteration limit: {n}", f"Precision: {d}", f"Computation time: {s} seconds.", sep = "\n")

## **The Newton-Raphson Method**
The Newton-Raphson Method, named after British mathematicians Isaac Newton (1642-1726/27) and Joseph Raphson (1668-1715), is a root-finding algorithm which seeks to approximate the roots of a real-valued function of a real variable using its first derivative. In particular, let $(a,b)$ be an ordered pair of real numbers in $\mathbb{R}^2$ such that $a < b$, let $f$ be a continuously differentiable function in $\mathscr{C}^{1}([a,b],\mathbb{R})$ such that $f^{-1}\left(\left\{0\right\}\right) \neq \emptyset$ and let $x_{\ast}$ be a real number in $f^{-1}\left(\left\{0\right\}\right)$. Then, for every real number $x$ in $[a,b]$, if the tangent to $\mathrm{graph}(f)$ at $x$ intersects $\mathbb{R} \times \{0\}$ at the point $(x_{\ast}, 0)$, then one has:
$$f^{\prime}(x) = \frac{f(x) - f(x_{\ast})}{x - x_{\ast}} = \frac{f(x) - 0}{x - x_{\ast}} = \frac{f(x)}{x - x_{\ast}}$$
whereupon:
$$f^{\prime}(x) \neq 0 \implies x_{\ast} = x - \frac{f(x)}{f^{\prime}(x)}$$
In particular, if $x_{0}$ is a real number in $[a,b]$, then, provided that the function $f$ is sufficietly well-behaved (e.g., $f^{\prime}$ is sufficiently non-vanishing) and $x_{0}$ is sufficiently close to $x_{\ast}$, then the sequence $(x_{n})_{n \in \mathbb{N}}$ defined by:
$$\begin{cases}x_{0} \in [a, b] \\ \forall n \in \mathbb{N}, \enspace x_{n + 1} = x_{n} - \dfrac{f(x_{n})}{f^{\prime}(x_{n})}\end{cases}$$
converges to $x_{\ast}$. The implementation of the Newton-Raphson method, as well as some performance metrics, are given in the code cell below.

In [15]:
def newton_raphson(f, df, x0, n = 100, d = 1e-6):
    f = np.vectorize(f)
    df = np.vectorize(df)
    x0 = np.array(x0)
    assert type(n) is int and n >= 1 and np.all(df(x0) != 0)
    for _ in range(n):
        x1 = np.subtract(x0, np.divide(f(x0), df(x0)))
        if np.all(np.abs(f(x1)) < d):
            break
        else:
            pass
        x0 = x1
    return np.array([x1, f(x1)])

newton_raphson.__name__ = "Newton-Raphson Method"

performance_metrics(lambda x: x**3, lambda x: 3 * x**2, [1, 2, 3], 100, 1e-6, method = newton_raphson)

Function: <function <lambda> at 0x000001C1A78ADAB0>
Method: Newton-Raphson Method
Result: [[2.28365826e-03 4.56731652e-03 6.85097478e-03]
 [1.19094949e-08 9.52759591e-08 3.21556362e-07]]
Iteration limit: 100
Precision: 1e-06
Computation time: 0.006 seconds.


## **The Secant Method**
The Secant Method is another root-finding algorithm which, although it does not use any derivative, instead uses secant lines passing through two particular points on the graph of a real-valued function of a real-variable. In particular, let $(a,b)$ be an ordered pair of real numbers in $\mathbb{R}^2$ such that $a < b$, let $f$ be a continuous function in $\mathscr{C}([a,b],\mathbb{R})$ such that $f^{-1}\left(\left\{0\right\}\right) \neq \emptyset$ and let $x_{\ast}$ be a real number in $f^{-1}\left(\left\{0\right\}\right)$. Then, for every ordered pair $(x_{0}, x_{1})$ of distinct real numbers $[a,b]^{2}$ such that $x_{0}$ \neq $x_{1}$, the secant to $\mathrm{graph}(f)$ at points $x_{0}$ and $x_{1}$ intersects $\mathbb{R} \times \{0\}$ at a point $(x_{2}, 0)$, satisfying:
$$\frac{f(x_{1}) - f(x_{0})}{x_{1} - x_{0}}(x_{2} - x_{0}) + f(x_{0}) = 0$$
whereupon:
$$f(x_{0}) \neq f(x_{1}) \implies x_{2} = \frac{x_{0}f(x_{1}) - x_{1}f(x_{0})}{f(x_{1})-f(x_{0})}$$
In particular, if $x_{0}$ and $x_{1}$ are distinct real numbers in $[a,b]$ such that $f(x_{0}) \neq f(x_{1})$, then, provided that the function $f$ is sufficietly well-behaved and $x_{0}$ and $x_{1}$ are sufficiently close to $x_{\ast}$, then the sequence $(x_{n})_{n \in \mathbb{N}}$ defined by:
$$\begin{cases}(x_{0}, x_{1}) \in [a, b]^2 \\ \forall n \in \mathbb{N}, \enspace x_{n + 2} = \dfrac{x_{n}f(x_{n+1}) - x_{n+1}f(x_{n})}{f(x_{n+1})-f(x_{n})}\end{cases}$$
converges to $x_{\ast}$. The implementation of the Secant Method, as well as some performance metrics, are given in the code cell below.

In [16]:
def secant(f, x0, x1, n = 100, d = 1e-6):
    f = np.vectorize(f)
    x0 = np.array(x0)
    x1 = np.array(x1)
    assert type(n) is int and n >= 1 and np.all(f(x0) != f(x1))
    for _ in range(n):
        x2 = np.divide(np.subtract(np.multiply(x0, f(x1)), np.multiply(x1, f(x0))), np.subtract(f(x1), f(x0)))
        if np.all(np.abs(f(x2)) < d):
            break
        else:
            pass
        x0 = x1
        x1 = x2
    return np.array([x2, f(x2)])

secant.__name__ = "Secant Method"

performance_metrics(lambda x: x**3, [1,2], [3,4], 100, 1e-6, method = secant)

Function: <function <lambda> at 0x000001C198FADE10>
Method: Secant Method
Result: [[5.10209540e-03 9.16331840e-03]
 [1.32814572e-07 7.69410897e-07]]
Iteration limit: 100
Precision: 1e-06
Computation time: 0.008 seconds.


## **Bisection Method**
The Bisection Method is another root-finding algorithm that uses the Intermediate-Value theorem. In particular, let $(a,b)$ be an ordered pair of real numbers in $\mathbb{R}^2$ such that $a < b$ and let $f$ be a continuous function in $\mathscr{C}([a,b], \mathbb{R})$ such that $f(a)f(b) < 0$. Then, by the intermediate value theorem, there exists a real number $x_{\ast}$ in $]a,b[$ such that $f(x_{\ast}) = 0$. In particular, $f^{-1}\left(\left\{0\right\}\right) \neq \emptyset$. The interval $[a,b]$ is successively halved, where the half searched at each step is the one known to contain at least one root of $f$. The implementation of the Bisection Method, as well as some performance metrics, are given in the code cell below.

In [17]:
def bisection(f, a, b, n = 100, d = 1e-6):
    f = np.vectorize(f)
    a = np.array(a)
    b = np.array(b)
    assert type(n) is int and n >= 1 and np.all(a < b) and np.all(f(a) * f(b) <= 0)
    for _ in range(n):
        c = np.divide(np.add(a, b), 2)
        if np.all(np.abs(f(c)) < d):
            break
        elif np.any(f(c) < -d):
            a = c
        else:
            b = c
    return np.array([c, f(c)])

bisection.__name__ = "Bisection Method"

performance_metrics(lambda x: x**3, [-1,-2], [1,2], 100, 1e-6, method = bisection)

Function: <function <lambda> at 0x000001C198FADD80>
Method: Bisection Method
Result: [[0. 0.]
 [0. 0.]]
Iteration limit: 100
Precision: 1e-06
Computation time: 0.0 seconds.


## **Credits**
- Wikipedia contributors, "[Newton's method](https://en.wikipedia.org/w/index.php?title=Newton%27s_method&oldid=1205325071)," Wikipedia, The Free Encyclopedia (accessed February 18, 2024).
- Wikipedia contributors, "[Secant method](https://en.wikipedia.org/w/index.php?title=Secant_method&oldid=1161824106)," Wikipedia, The Free Encyclopedia (accessed February 18, 2024).
- Wikipedia contributors, "[Bisection method](https://en.wikipedia.org/w/index.php?title=Bisection_method&oldid=1189644973)," Wikipedia, The Free Encyclopedia (accessed February 18, 2024).
- Wikipedia contributors, "[Isaac Newton](https://en.wikipedia.org/w/index.php?title=Isaac_Newton&oldid=1207573075)," Wikipedia, The Free Encyclopedia (accessed February 18, 2024).
- Wikipedia contributors, "[Joseph Raphson](https://en.wikipedia.org/w/index.php?title=Joseph_Raphson&oldid=1174227174)," Wikipedia, The Free Encyclopedia (accessed February 18, 2024).
- The original code is provided as-is in this project by Zakaria Zerrouki.